<a href="https://colab.research.google.com/github/Morsalah/M.Sc-Research-HRI-using-DIGIT-tactile-sensor/blob/main/capture_image_auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import logging
import csv
import cv2
import numpy as np
from digit_interface.digit import Digit

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Function to create directories
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)
        logger.info(f"Directory {path} created.")
    else:
        logger.info(f"Directory {path} already exists.")

# Function to compare images and determine if touch is detected
def is_touch_detected(baseline_image, new_image, threshold=50):
    """
    Compare the new image with the baseline image.
    If the difference exceeds the threshold, classify it as 'touch detected'.
    """
    # Convert images to grayscale
    baseline_gray = cv2.cvtColor(baseline_image, cv2.COLOR_BGR2GRAY)
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Compute absolute difference
    diff = cv2.absdiff(baseline_gray, new_gray)

    # Calculate mean intensity of differences
    diff_score = np.mean(diff)

    logger.debug(f"Difference score: {diff_score}")

    # If the difference is above threshold, it's a touch (YES)
    return diff_score > threshold

# Capture and auto-label images
def capture_and_save_images(serial: str, save_path: str, num_images: int = 100, interval: float = 1.0):
    yes_path = os.path.join(save_path, "YES")
    no_path = os.path.join(save_path, "NO")
    create_directory(yes_path)
    create_directory(no_path)

    csv_filename = os.path.join(save_path, "labels.csv")
    csv_exists = os.path.exists(csv_filename)

    with open(csv_filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        if not csv_exists:
            writer.writerow(["filename", "label"])  # Add header if file is new

        # Connect to the DIGIT device
        digit = Digit(serial)
        digit.connect()

        # Capture baseline image (No Touch)
        print("Please ensure there is NO TOUCH for baseline image capture...")
        time.sleep(2)
        baseline_filename = os.path.join(save_path, "baseline.png")
        digit.save_frame(baseline_filename)
        baseline_image = cv2.imread(baseline_filename)
        logger.info(f"Baseline image saved: {baseline_filename}")

        yes_count, no_count = 0, 0

        for i in range(num_images):
            try:
                # Capture new frame
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                image_filename = f"{save_path}/digit_{serial}_{timestamp}_{i+1}.png"
                digit.save_frame(image_filename)

                # Read the newly captured image
                new_image = cv2.imread(image_filename)

                # Auto-label based on comparison
                if is_touch_detected(baseline_image, new_image):
                    label = "yes"
                    save_folder = yes_path
                    yes_count += 1
                else:
                    label = "no"
                    save_folder = no_path
                    no_count += 1

                # Move image to correct folder
                new_image_path = os.path.join(save_folder, os.path.basename(image_filename))
                os.rename(image_filename, new_image_path)

                # Log filename and label in CSV
                writer.writerow([new_image_path, label])
                logger.info(f"Image saved: {new_image_path} | Label: {label}")

                # Stop if 50 images per category are collected
                if yes_count >= num_images // 2 and no_count >= num_images // 2:
                    print("Collected required number of images for both categories.")
                    break

                # Wait for the specified interval
                time.sleep(interval)

            except Exception as e:
                logger.error(f"Error capturing image {i+1}: {e}")

        # Disconnect the device
        digit.disconnect()

if __name__ == "__main__":
    # Define parameters
    serial_number = "D21115"
    save_directory = "./captured_images"
    total_images = 100  # 50 YES and 50 NO images

    # Start capturing images with auto-labeling
    capture_and_save_images(serial_number, save_directory, num_images=total_images)


In [ ]:
import os
import time
import logging
import csv
import cv2
import numpy as np
from digit_interface.digit import Digit

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Function to create directories
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)
        logger.info(f"Directory {path} created.")
    else:
        logger.info(f"Directory {path} already exists.")

# Function to compare images and determine if touch is detected
def is_touch_detected(baseline_image, new_image, threshold=50):
    """
    Compare the new image with the baseline image.
    If the difference exceeds the threshold, classify it as 'touch detected'.
    """
    # Convert images to grayscale
    baseline_gray = cv2.cvtColor(baseline_image, cv2.COLOR_BGR2GRAY)
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Compute absolute difference
    diff = cv2.absdiff(baseline_gray, new_gray)

    # Calculate mean intensity of differences
    diff_score = np.mean(diff)

    logger.debug(f"Difference score: {diff_score}")

    # If the difference is above threshold, it's a touch (YES)
    return diff_score > threshold

# Capture and auto-label images
def capture_and_save_images(serial: str, save_path: str, num_images: int = 100, interval: float = 1.0):
    yes_path = os.path.join(save_path, "YES")
    no_path = os.path.join(save_path, "NO")
    create_directory(yes_path)
    create_directory(no_path)

    csv_filename = os.path.join(save_path, "labels.csv")
    csv_exists = os.path.exists(csv_filename)

    with open(csv_filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        if not csv_exists:
            writer.writerow(["filename", "label"])  # Add header if file is new

        # Connect to the DIGIT device
        digit = Digit(serial)
        digit.connect()

        # Capture baseline image (No Touch)
        print("Please ensure there is NO TOUCH for baseline image capture...")
        time.sleep(2)
        baseline_filename = os.path.join(save_path, "baseline.png")
        digit.save_frame(baseline_filename)
        baseline_image = cv2.imread(baseline_filename)
        logger.info(f"Baseline image saved: {baseline_filename}")

        yes_count, no_count = 0, 0

        for i in range(num_images):
            try:
                # Capture new frame
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                image_filename = f"{save_path}/digit_{serial}_{timestamp}_{i+1}.png"
                digit.save_frame(image_filename)

                # Read the newly captured image
                new_image = cv2.imread(image_filename)

                # Auto-label based on comparison
                if is_touch_detected(baseline_image, new_image):
                    label = "yes"
                    save_folder = yes_path
                    yes_count += 1
                else:
                    label = "no"
                    save_folder = no_path
                    no_count += 1

                # Move image to correct folder
                new_image_path = os.path.join(save_folder, os.path.basename(image_filename))
                os.rename(image_filename, new_image_path)

                # Log filename and label in CSV
                writer.writerow([new_image_path, label])
                logger.info(f"Image saved: {new_image_path} | Label: {label}")

                # Stop if 50 images per category are collected
                if yes_count >= num_images // 2 and no_count >= num_images // 2:
                    print("Collected required number of images for both categories.")
                    break

                # Wait for the specified interval
                time.sleep(interval)

            except Exception as e:
                logger.error(f"Error capturing image {i+1}: {e}")

        # Disconnect the device
        digit.disconnect()

if __name__ == "__main__":
    # Define parameters
    serial_number = "D21115"
    save_directory = "./captured_images"
    total_images = 100  # 50 YES and 50 NO images

    # Start capturing images with auto-labeling
    capture_and_save_images(serial_number, save_directory, num_images=total_images)


In [ ]:
import os
import time
import logging
import csv
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from digit_interface.digit import Digit

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Function to create directories
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)
        logger.info(f"Directory {path} created.")

# Function to compare images using SSIM + Pixel Difference
def is_touch_detected(baseline_image, new_image, ssim_threshold=0.8, pixel_change_threshold=5):
    """
    Uses both SSIM and pixel intensity difference to determine touch detection.
    """
    # Convert images to grayscale
    baseline_gray = cv2.cvtColor(baseline_image, cv2.COLOR_BGR2GRAY)
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Compute Structural Similarity Index (SSIM)
    similarity_score = ssim(baseline_gray, new_gray)

    # Compute absolute pixel difference
    diff = cv2.absdiff(baseline_gray, new_gray)
    changed_pixels = np.sum(diff > 20)  # Count pixels with significant change (threshold 20)
    total_pixels = diff.size
    pixel_change_percent = (changed_pixels / total_pixels) * 100

    logger.debug(f"SSIM score: {similarity_score}, Pixel change: {pixel_change_percent:.2f}%")

    # If SSIM is low OR pixel difference is high, classify as touch
    return similarity_score < ssim_threshold or pixel_change_percent > pixel_change_threshold

# Capture and auto-label images
def capture_and_save_images(serial: str, save_path: str, num_images: int = 100, interval: float = 1.0):
    yes_path = os.path.join(save_path, "YES")
    no_path = os.path.join(save_path, "NO")
    create_directory(yes_path)
    create_directory(no_path)

    csv_filename = os.path.join(save_path, "labels.csv")
    csv_exists = os.path.exists(csv_filename)

    with open(csv_filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        if not csv_exists:
            writer.writerow(["filename", "label"])  # Add header if file is new

        # Connect to the DIGIT device
        digit = Digit(serial)
        digit.connect()

        # Capture multiple baseline images (No Touch)
        print("Ensure NO TOUCH for baseline image capture...")
        time.sleep(2)
        baseline_images = []
        for _ in range(3):  # Capture 3 baseline images for better reference
            baseline_filename = os.path.join(save_path, f"baseline_{_}.png")
            digit.save_frame(baseline_filename)
            baseline_images.append(cv2.imread(baseline_filename))
            time.sleep(0.5)

        # Compute average baseline image
        baseline_image = np.mean(np.array(baseline_images), axis=0).astype(np.uint8)
        logger.info("Baseline images captured and averaged.")

        yes_count, no_count = 0, 0

        for i in range(num_images):
            try:
                # Capture new frame
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                image_filename = f"{save_path}/digit_{serial}_{timestamp}_{i+1}.png"
                digit.save_frame(image_filename)

                # Read the newly captured image
                new_image = cv2.imread(image_filename)

                # Auto-label based on SSIM + Pixel Difference
                if is_touch_detected(baseline_image, new_image):
                    label = "yes"
                    save_folder = yes_path
                    yes_count += 1
                else:
                    label = "no"
                    save_folder = no_path
                    no_count += 1

                # Move image to correct folder
                new_image_path = os.path.join(save_folder, os.path.basename(image_filename))
                os.rename(image_filename, new_image_path)

                # Log filename and label in CSV
                writer.writerow([new_image_path, label])
                logger.info(f"Image saved: {new_image_path} | Label: {label}")

                # Stop if 50 images per category are collected
                if yes_count >= num_images // 2 and no_count >= num_images // 2:
                    print("Collected required number of images for both categories.")
                    break

                # Wait for the specified interval
                time.sleep(interval)

            except Exception as e:
                logger.error(f"Error capturing image {i+1}: {e}")

        # Disconnect the device
        digit.disconnect()

if __name__ == "__main__":
    # Define parameters
    serial_number = "D21115"
    save_directory = "./captured_images"
    total_images = 100  # 50 YES and 50 NO images

    # Start capturing images with auto-labeling
    capture_and_save_images(serial_number, save_directory, num_images=total_images)
